In [1]:
import requests
import json
import pandas as pd

In [4]:
def refresh_headers():
    r = requests.post('https://accounts.spotify.com/api/token', data={
        'grant_type': 'refresh_token',
        'refresh_token': 'AQBUMI3FVqVNmfmxpWgBb4aGNZ6BzT7mf85XoC8Aps4K_RJnnqNm_aajnLOVIqnEOa5a_Qfg-n-a_axaVMr9gtF2205hvi9mi1IFT3n5Iep7DrLKtl5gl5T9RZ-_tmqTXiU',
        'client_id': '908d3e07450148849b073e188ba0216a',
        'client_secret': 'd14f7cf76e734ba1b32e43b79f5e48cb'
    })
    return {'Authorization': 'Bearer {}'.format(json.loads(r.content)['access_token']),
           'Content-Type': 'application/json'}

In [5]:
headers = refresh_headers()

## Top tracks and artists

In [9]:
typ = 'tracks'
time_range = 'long_term'
limit = 10

In [10]:
r = requests.get(f'https://api.spotify.com/v1/me/top/{typ}?time_range={time_range}&limit={limit}', headers=headers)

In [11]:
list(map(lambda f: (f['name'], f['popularity'], list(map(lambda a: a['name'], f['artists']))), json.loads(r.content)['items']))

[('+', 63, ['Aitana', 'Cali Y El Dandee']),
 ('High', 46, ['Maria Becerra']),
 ('Dejemos Que Pase', 39, ['Maria Becerra']),
 ('Mi Suerte', 70, ['Morat']),
 ('Perdón por las Horas', 48, ['Pol Granch']),
 ('Le chant des sirènes 2', 37, ['Fréro Delavega']),
 ('Nada de Amor', 35, ['Maria Becerra']),
 ('Mi Debilidad', 66, ['Maria Becerra']),
 ('Me Gusta', 56, ['Nicki Nicole']),
 ('Desastre', 45, ['Pol Granch'])]

## Playlists

In [2]:
user_id = '11182136211'

In [6]:
r = json.loads(requests.get(f'http://api.spotify.com/v1/me/playlists?limit=50', headers=headers).content)

In [7]:
oks = list(filter(lambda f: f['owner']['id']==user_id and f['tracks']['total']>=30, r['items']))

In [8]:
list(map(lambda f: (f['name'], f['tracks']['total']), oks))

[('Peligrosa', 280),
 ('popsicle', 47),
 ('yes', 55),
 ('ruteros', 43),
 ('yay', 76),
 ('pv2', 575),
 ('le fr', 39),
 ('Imagination', 59),
 ('forest walk', 81),
 ('tom & two', 52),
 ('Muy Tranquilo', 153),
 ('movies', 34),
 ("c'est la vie", 92),
 ('laxin', 34)]

In [12]:
playlist_map = {f['id']: f['name'] for f in oks if f['name'] in [
    'Peligrosa','pv2','popsicle','forest walk','Muy Tranquilo'
]}

In [13]:
playlist_map

{'0x7qUjuFBkaNO5xYpWG4MI': 'Peligrosa',
 '28fZyyGjM0Yb8ZSITmdi0j': 'popsicle',
 '2HJbwEyjWXKyqeXpjdExUd': 'pv2',
 '4y8KPM29YuzdnPX6iuExfH': 'forest walk',
 '6UgUBdv2BqkEzvxaKpOIoi': 'Muy Tranquilo'}

In [33]:
data = []
i = 0
total = 1
latest_n = 30
while i < total:
    if latest_n is not None:
        batch_size = latest_n
    raw_data = json.loads(requests.get(
        f'https://api.spotify.com/v1/playlists/0x7qUjuFBkaNO5xYpWG4MI/tracks?limit={batch_size}&offset={i}&fields=total,items(added_at,track(id,name,popularity))',
        headers=headers
    ).content)
    data.extend(list(map(lambda f: {
        'track_id': f['track']['id'], 'added_at': f['added_at'], 'name': f['track']['name'], 'popularity': f['track']['popularity']
    }, raw_data['items'])))
    total = raw_data['total']
    if latest_n is not None and i == 0:
        i = total - batch_size
        data = []
    else:
        i += batch_size

In [34]:
data

[{'track_id': '6Sq7ltF9Qa7SNFBsV5Cogx',
  'added_at': '2022-05-09T01:59:00Z',
  'name': 'Me Porto Bonito',
  'popularity': 99},
 {'track_id': '1txrFHh4XUU3Kp8HcD2TFW',
  'added_at': '2022-05-09T02:01:58Z',
  'name': 'Te Mentiría Remix',
  'popularity': 77},
 {'track_id': '3zUtuSPwyqKElkqfnJj3X2',
  'added_at': '2022-05-09T02:11:07Z',
  'name': 'NO MIENTEN',
  'popularity': 69},
 {'track_id': '41oY4WCTj5kccfesTVFnvN',
  'added_at': '2022-05-09T02:29:54Z',
  'name': 'Tarot',
  'popularity': 94},
 {'track_id': '7kYbxvrGXv8cmKjkqgqhrw',
  'added_at': '2022-05-11T01:18:39Z',
  'name': 'La Triple T',
  'popularity': 84},
 {'track_id': '78SjIkujtNsOewFNlrlr08',
  'added_at': '2022-05-11T16:29:02Z',
  'name': 'Sonrisa Falsa (with Sael)',
  'popularity': 48},
 {'track_id': '6LRzzkbOtYNPrlhlcjBUCs',
  'added_at': '2022-05-16T19:40:33Z',
  'name': 'Hermosa (feat. Daddy Yankee)',
  'popularity': 50},
 {'track_id': '7INDea2qgB7TlUgpux3hsg',
  'added_at': '2022-05-18T17:03:35Z',
  'name': 'Castigo',

In [25]:
len(data)

47

In [14]:
data = json.loads(requests.get(
    f'https://api.spotify.com/v1/playlists/0x7qUjuFBkaNO5xYpWG4MI/tracks?limit=10&offset=0&fields=total,items(added_at,track(id,name))',
    headers=headers
).content)

In [15]:
data

{'items': [{'added_at': '2020-04-14T16:38:40Z',
   'track': {'id': '6LnXhWRXn8i7xUjEaqwIE9', 'name': 'Dejemos Que Pase'}},
  {'added_at': '2020-04-14T16:38:48Z',
   'track': {'id': '5vPtzdjJ9tm7XCoKDH22Gb', 'name': 'High'}},
  {'added_at': '2020-04-14T16:38:56Z',
   'track': {'id': '5utL6qFkMGP7EOw6Gri5wi', 'name': 'Nada de Amor'}},
  {'added_at': '2020-04-14T16:39:04Z',
   'track': {'id': '2cn7dsEVaIHslGDMWOPlW6', 'name': 'Fucking Diablo'}},
  {'added_at': '2020-04-14T16:39:11Z',
   'track': {'id': '4xT8BKG51PWefWHmPnm7Wc', 'name': 'Me Gusta'}},
  {'added_at': '2020-04-14T16:39:30Z',
   'track': {'id': '6kyp35INigDcwXHTKCHoAk', 'name': 'Diva'}},
  {'added_at': '2020-04-14T16:51:45Z',
   'track': {'id': '2032dz8FNUftQh5NVjmCvz', 'name': 'Culpa'}},
  {'added_at': '2020-04-15T13:15:48Z',
   'track': {'id': '5xCGpVb4IYAREVhyT2zqyH', 'name': 'Peligrosa (Mimosas)'}},
  {'added_at': '2020-04-17T23:21:39Z',
   'track': {'id': '7aFBy1CBLGnZE1Iv6TvEZE',
    'name': 'Antes de Morirme (feat. Rosa

#### Playlists tracks

In [569]:
playlist_tracks = {}
for p_id in playlist_map.keys():
    playlist_tracks[p_id] = {}
    i = 0
    total = 1
    while i < total:
        data = json.loads(requests.get(
            f'https://api.spotify.com/v1/playlists/{p_id}/tracks?limit=50&offset={i}&fields=total,items.track(id,name)',
            headers=headers
        ).content)
        playlist_tracks[p_id].update({t['track']['id']: t['track']['name'] for t in data['items']})
        total = data['total']
        i += 50

In [576]:
data['items'][0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

#### Tracks audio features

In [518]:
features = {}
for p_id, track_data in playlist_tracks.items():
    track_ids_str = ','.join(list(track_data.keys())[-50:])
    features[p_id] = json.loads(requests.get(
        f'https://api.spotify.com/v1/audio-features/?ids={track_ids_str}', headers=headers
    ).content)['audio_features']
    for feats in features[p_id]:
        feats['name'] = track_data[feats['id']]

### Process data

In [553]:
df = pd.DataFrame(columns=list(list(features.values())[0][0].keys())+['playlist'])
i = 0
for p_id, tracks in features.items():
    for track in tracks:
        df.loc[i] = {**track, 'playlist': p_id}
        i+=1
df.drop(['track_href','analysis_url','duration_ms','uri','type','time_signature'], axis=1, inplace=True)
df['playlist'] = df['playlist'].map(playlist_map)

In [554]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,name,playlist
0,0.859,0.407,1,-6.317,0,0.0460,0.0990,0,0.1750,0.808,88.039,1yihMXFt0zsG86nue7u41U,Dime,Peligrosa
1,0.902,0.556,11,-4.109,1,0.1720,0.2470,0,0.0951,0.712,90.037,4jozbMEic7fUM2xH9P90Oy,Pal Perreo,Peligrosa
2,0.830,0.739,7,-5.395,0,0.0627,0.1360,0.000002,0.1620,0.631,97.993,4srcMdbm2qzVG98tJajHxp,Pantysito,Peligrosa
3,0.834,0.670,0,-4.324,1,0.1480,0.1060,0.000035,0.0836,0.612,97.964,4Z6ucGPXIzZZTN70Q7KQM0,Gatas,Peligrosa
4,0.826,0.618,10,-4.848,0,0.1060,0.1670,0,0.1140,0.537,87.958,5idykJt11B56B8XdeDg3Tj,Nadie,Peligrosa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,0.646,0.455,6,-9.563,1,0.0309,0.0589,0.747,0.3660,0.423,139.944,2vpKeYpjTmzR4RfrLUYm5n,Azure - Emancipator Remix,Muy Tranquilo
243,0.663,0.465,11,-11.532,0,0.0300,0.0542,0.945,0.1110,0.271,125.516,7CGhUAQcsn81EVNqCHfxGc,Sea To Sky,Muy Tranquilo
244,0.712,0.685,11,-8.431,0,0.0340,0.4400,0.865,0.0975,0.059,104.980,3MZoVNwQG5SCfLqGtpbbJt,Sea To Sky - Parra For Cuva Remix,Muy Tranquilo
245,0.751,0.670,4,-7.883,0,0.0633,0.2390,0.0515,0.4270,0.203,119.998,2Nsz5vf1G6jxQs9JdKkTaT,Sirens,Muy Tranquilo


#### Extract 2 components

In [555]:
import matplotlib.pyplot as plt
import plotly_express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [556]:
cols = ['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness',
        'valence','tempo']

In [557]:
n_components = 2
scaler = StandardScaler()
pca = PCA(n_components=n_components)
new_features = pd.DataFrame(
    pca.fit_transform(scaler.fit_transform(df[cols])),
    columns=[f'Feature{i+1}' for i in range(n_components)]
)

In [558]:
for col in new_features.columns:
    df[col] = new_features[col]

In [559]:
df['color'] = df['playlist'].map(dict(zip(df['playlist'].unique(),['r','g','b','y'])))

In [560]:
fig = px.scatter(data_frame=df, x='Feature1', y='Feature2', color='playlist', custom_data=['name'])
fig.update_traces(hovertemplate="%{customdata[0]}")

In [561]:
pca.explained_variance_ratio_

array([0.32675063, 0.13788587])

In [567]:
fig = make_subplots(rows=len(cols), cols=1)
for i, col in enumerate(cols):
    fig.add_trace(go.Box(x=df['playlist'], y=df[col], name=col), row=i+1, col=1)

fig.update_layout(height=2000, width=800, title_text="Features for different playlists")
fig.show()

#### Clustering

In [445]:
clusters = 4
km = KMeans(n_clusters=clusters)
df['cluster'] = km.fit_predict(df[cols])

In [446]:
fig = px.scatter(data_frame=df, x='Feature1', y='Feature2', color=df['cluster'].astype(str), custom_data=['name'])
fig.update_traces(hovertemplate="%{customdata[0]}")

In [447]:
d = pd.DataFrame(columns=range(clusters), index=range(150))
for i in range(clusters):
    a = df.loc[df['cluster']==i, 'name'].values
    d.loc[:len(a)-1, i] = a

In [448]:
d.head(35)

,0,1,2,3
0,Vdp,Dime,Pal Perreo,A carta cabal
1,Real Friends,Pantysito,Si Te Interesa,Permanent Way
2,This Love,Gatas,Si Quieren Frontear,Rise
3,In the Dark,Nadie,TBT,To Build A Home
4,The Climb,Sin Ropa (with Ingratax),Instagram,San Luis
5,The One That Got Away - Acoustic,cuatro veinte,Te Felicito,NaN
6,feel something,Cucando,NO MIENTEN,NaN
7,Tu cepillo de dientes,Mojando Asientos (feat. Feid),Castigo,NaN
8,Tierra Firme,Tendencia Global,Nota,NaN
9,Cosas De La Vida,No Son Horas,Barranquilla Bajo Cero,NaN


In [453]:
fig = make_subplots(rows=len(cols), cols=1)
for i, col in enumerate(cols):
    fig.add_trace(go.Box(x=df['cluster'], y=df[col], name=col), row=i+1, col=1)

fig.update_layout(height=2000, width=800, title_text="Side By Side Subplots")
fig.show()